## Парсим сообщения из telegram каналов

### Импорты

In [1]:
import os

import pandas as pd
import re
import time
from tqdm.notebook import tqdm
from datetime import date, datetime

from telethon import TelegramClient
import config

import json

import warnings
warnings.filterwarnings('ignore')

### Создаем клиент Telegram API

In [2]:
api_id = config.API_ID
api_hash = config.API_HASH
name = 'test'

In [3]:
client = TelegramClient(name, api_id, api_hash)
await client.start()

### Парсинг сообщений

In [4]:
channels_info = pd.read_csv('../../../data/telegram/channels.csv')

In [5]:
channels_info.head()

,channel_id,name,subscribers,link,description,create_date
0,0,Тинькофф Инвестиции,670825,https://t.me/tinkoff_invest_official,Официальный канал Тинькофф.,2019-09-04 15:50:32
1,1,Альфа-Инвестиции,415942,https://t.me/alfa_investments,Официальный канал Альфа-Инвестиций с инвестиде...,2021-06-18 14:04:01
2,2,Мои Инвестиции,286117,https://t.me/omyinvestments,"Инвестидеи, обзоры рынка и многое другое от лу...",2019-09-12 12:53:25
3,3,bitkogan,284405,https://t.me/bitkogan,Авторский канал Евгения Когана,2018-01-14 14:46:07
4,4,СберИнвестиции,232751,https://t.me/SberInvestments,"Канал СберИнвестиции, в котором мы рассказывае...",2020-07-24 16:15:43


In [6]:
from telethon.tl.functions.messages import GetHistoryRequest

async def dump_all_messages(channel):
    
    offset_msg = 0    # номер записи, с которой начинается считывание
    limit_msg = 100   # максимальное число записей, передаваемых за один раз
    all_messages = []   # список всех сообщений
    total_messages = 0

    while True:
        history = await client(GetHistoryRequest(
            peer=channel,
            offset_id=offset_msg,
            offset_date=None, add_offset=0,
            limit=limit_msg, max_id=0, min_id=0,
            hash=0))
        
        if not history.messages:
            break
            
        messages = history.messages
        
        for message in messages:
            all_messages.append(message.to_dict())
            
        offset_msg = messages[len(messages) - 1].id
        
    msg_cnt = len(all_messages)
        
    return all_messages, msg_cnt

In [7]:
messages_dict = dict()

In [8]:
for i, channel in enumerate(tqdm(channels_info.to_dict('records'))):
    
    messages_dict[channel['channel_id']] = dict()
    try:
        url = channel['link']
        tg_channel = await client.get_entity(url)
        cur_channel_messages, msg_cnt = await dump_all_messages(tg_channel)

        messages_dict[channel['channel_id']]['messages'] = cur_channel_messages
        messages_dict[channel['channel_id']]['msg_cnt'] = msg_cnt
        messages_dict[channel['channel_id']]['error'] = None

    except Exception as ex:
        pass
        # messages_dict[channel['channel_id']]['messages'] = []
        # messages_dict[channel['channel_id']]['msg_cnt'] = 0
        # messages_dict[channel['channel_id']]['error'] = ex.message
        
    time.sleep(1)

  0%|          | 0/71 [00:00<?, ?it/s]

Telegram is having internal issues RpcCallFailError: Telegram is having internal issues, please try again later. (caused by GetHistoryRequest)
Telegram is having internal issues RpcCallFailError: Telegram is having internal issues, please try again later. (caused by GetHistoryRequest)


In [9]:
from sys import getsizeof

print(f'{round(getsizeof(messages_dict)/1024, 3)}KB')

2.219KB


Посмотрим на пример сообщения

In [13]:
messages_dict[0]['messages'][3]

{'_': 'Message',
 'id': 6263,
 'peer_id': {'_': 'PeerChannel', 'channel_id': 1344086554},
 'date': datetime.datetime(2024, 3, 22, 16, 43, 51, tzinfo=datetime.timezone.utc),
 'message': '',
 'out': False,
 'mentioned': False,
 'media_unread': False,
 'silent': False,
 'post': True,
 'from_scheduled': False,
 'legacy': False,
 'edit_hide': False,
 'pinned': False,
 'noforwards': False,
 'from_id': None,
 'fwd_from': None,
 'via_bot_id': None,
 'reply_to': None,
 'media': {'_': 'MessageMediaPhoto',
  'photo': {'_': 'Photo',
   'id': 5472065816217376882,
   'access_hash': -8854135167567968593,
   'file_reference': b'\x02P\x1d"\x1a\x00\x00\x18wf\x00w \xb9|\xab\xd8\x13\xef\xa3K\x84\xda\xe4%j\t,A',
   'date': datetime.datetime(2024, 3, 22, 16, 5, 54, tzinfo=datetime.timezone.utc),
   'sizes': [{'_': 'PhotoStrippedSize',
     'type': 'i',
     'bytes': b'\x01((\xa8\x88\x85r\xd2\x05>\x984\xbe\\\x7f\xf3\xdc\x7f\xdf&\xa3\x1d?\x87\xf1\xa4>\xbcS\x01\xec\x91\x80q.O\xfb\xa6\xa2\xabQ\x91\xe5\x0f\xf5=;

### Подготовка данных

In [14]:
messages_df = pd.DataFrame(columns = ['channel_id', 'tg_message_id', 'date', 'text', 'views', 'forwards', 'reactions'])

In [15]:
for key in tqdm(messages_dict.keys()):
    try:
        for i, _ in enumerate(messages_dict[key]['messages']):

            row_dict = {
                    'channel_id' : key,
                    'tg_message_id' : messages_dict[key]['messages'][i]['id'],
                    'date' : pd.to_datetime(messages_dict[key]['messages'][i]['date']).tz_localize(None),
                    'text' : messages_dict[key]['messages'][i]['message'],
                    'views' : messages_dict[key]['messages'][i]['views'],
                    'forwards' : messages_dict[key]['messages'][i]['forwards'],
                    'reactions' : None
                }

            try:
                row_dict['reactions'] = messages_dict[key]['messages'][i]['reactions']['results']
            except:
                pass

            row_df = pd.DataFrame([row_dict]) 
            messages_df = pd.concat([messages_df, row_df], ignore_index = True)

    except:
        pass
        

  0%|          | 0/71 [00:00<?, ?it/s]

Server sent a very old message with ID 7350023071498024961, ignoring
Server sent a very old message with ID 7350023083583013889, ignoring
Server sent a very old message with ID 7350023098326418433, ignoring
Server sent a very old message with ID 7350023126181456897, ignoring
Server sent a very old message with ID 7350023133826536449, ignoring
Server sent a very old message with ID 7350023142622389249, ignoring
Server sent a very old message with ID 7350023162928778241, ignoring
Server sent a very old message with ID 7350023163518251009, ignoring
Server sent a very old message with ID 7350023175140750337, ignoring


In [18]:
messages_df = messages_df[messages_df['text'] != '']
messages_df = messages_df.reset_index().rename({'index' : 'message_id'}, axis = 1)
messages_df.head()

,message_id,channel_id,tg_message_id,date,text,views,forwards,reactions
0,4,0,6262,2024-03-22 16:43:50,📊 Как закрылась основная торговая сессия на ро...,164587,21,"[{'_': 'ReactionCount', 'reaction': '👍', 'coun..."
1,5,0,6261,2024-03-22 16:05:03,"​​​​​​​​​😑 Надоело: Америка растет, Россия — н...",100065,70,"[{'_': 'ReactionCount', 'reaction': '😁', 'coun..."
2,6,0,6260,2024-03-22 14:45:18,Подводим итоги недели традиционным дайджестом ...,140765,19,"[{'_': 'ReactionCount', 'reaction': '👍', 'coun..."
3,7,0,6258,2024-03-22 14:00:08,​​Ставка сохранена. Как быть?\nПодключайтесь к...,83494,3,"[{'_': 'ReactionCount', 'reaction': '👍', 'coun..."
4,8,0,6257,2024-03-22 12:57:06,Сплит акций Норильского никеля уже близко\nСам...,89094,148,"[{'_': 'ReactionCount', 'reaction': '👍', 'coun..."


In [19]:
messages_cnt = messages_df.groupby('channel_id').count()['message_id'].reset_index().rename({'message_id' : 'message_cnt'}, axis=1)

In [20]:
channels_info = channels_info.merge(messages_cnt, how = 'left', left_on = 'channel_id', right_on = 'channel_id')
channels_info.head()

,channel_id,name,subscribers,link,description,create_date,message_cnt
0,0,Тинькофф Инвестиции,670825,https://t.me/tinkoff_invest_official,Официальный канал Тинькофф.,2019-09-04 15:50:32,44.0
1,1,Альфа-Инвестиции,415942,https://t.me/alfa_investments,Официальный канал Альфа-Инвестиций с инвестиде...,2021-06-18 14:04:01,384.0
2,2,Мои Инвестиции,286117,https://t.me/omyinvestments,"Инвестидеи, обзоры рынка и многое другое от лу...",2019-09-12 12:53:25,11.0
3,3,bitkogan,284405,https://t.me/bitkogan,Авторский канал Евгения Когана,2018-01-14 14:46:07,1777.0
4,4,СберИнвестиции,232751,https://t.me/SberInvestments,"Канал СберИнвестиции, в котором мы рассказывае...",2020-07-24 16:15:43,419.0


Итак, получены все сообщения для каждого telegram канала из нашего списка

Оставим только сообщения с каналов, где больше 100 сообщений

In [22]:
channels_info.to_csv('../../../data/telegram/channels.csv', index = False)

In [23]:
messages_df = messages_df.merge(messages_cnt, how = 'left', left_on = 'channel_id', right_on = 'channel_id')
messages_df.head()

,message_id,channel_id,tg_message_id,date,text,views,forwards,reactions,message_cnt
0,4,0,6262,2024-03-22 16:43:50,📊 Как закрылась основная торговая сессия на ро...,164587,21,"[{'_': 'ReactionCount', 'reaction': '👍', 'coun...",44
1,5,0,6261,2024-03-22 16:05:03,"​​​​​​​​​😑 Надоело: Америка растет, Россия — н...",100065,70,"[{'_': 'ReactionCount', 'reaction': '😁', 'coun...",44
2,6,0,6260,2024-03-22 14:45:18,Подводим итоги недели традиционным дайджестом ...,140765,19,"[{'_': 'ReactionCount', 'reaction': '👍', 'coun...",44
3,7,0,6258,2024-03-22 14:00:08,​​Ставка сохранена. Как быть?\nПодключайтесь к...,83494,3,"[{'_': 'ReactionCount', 'reaction': '👍', 'coun...",44
4,8,0,6257,2024-03-22 12:57:06,Сплит акций Норильского никеля уже близко\nСам...,89094,148,"[{'_': 'ReactionCount', 'reaction': '👍', 'coun...",44


In [26]:
messages_df = messages_df.query('message_cnt >= 100').reset_index(drop=True).drop('message_cnt', axis = 1).sort_values(['channel_id', 'date'])
messages_df.head()

,message_id,channel_id,tg_message_id,date,text,views,forwards,reactions
383,552,1,4332,2023-11-02 12:31:28,"Центробанку, скорее всего, придётся поднять ст...",110394,401,"[{'_': 'ReactionCount', 'reaction': '👍', 'coun..."
382,551,1,4333,2023-11-02 14:33:01,Сбербанк продолжает рост: результаты по междун...,110495,76,"[{'_': 'ReactionCount', 'reaction': '👍', 'coun..."
381,550,1,4337,2023-11-02 15:56:40,Плохие новости сегодня вечером. СПБ Биржа попа...,125490,469,"[{'_': 'ReactionCount', 'reaction': '😢', 'coun..."
380,549,1,4338,2023-11-03 07:59:27,📉 Вчера на российском рынке акций прошли распр...,323274,41,"[{'_': 'ReactionCount', 'reaction': '👍', 'coun..."
379,548,1,4339,2023-11-03 10:44:50,У Норникеля сразу две приятные новости: компан...,315471,240,"[{'_': 'ReactionCount', 'reaction': '👍', 'coun..."


In [27]:
path = '../../../data/telegram/messages.csv'

messages_df.to_csv(path, index=False)

Server sent a very old message with ID 7350109406813263873, ignoring
Server sent a very old message with ID 7350109988490213377, ignoring
Server sent a very old message with ID 7350110308637861889, ignoring
Server sent a very old message with ID 7350160402570457089, ignoring
Server sent a very old message with ID 7350160745009928193, ignoring
Server sent a very old message with ID 7350160756049949697, ignoring
Server sent a very old message with ID 7350160765877806081, ignoring
Server sent a very old message with ID 7350161006335710209, ignoring
Server sent a very old message with ID 7350161173802379265, ignoring
Server sent a very old message with ID 7350161213388895233, ignoring
Server sent a very old message with ID 7350161255951626241, ignoring
Server sent a very old message with ID 7350161352218076161, ignoring
Server sent a very old message with ID 7350161506605497345, ignoring
Security error while unpacking a received message: Too many messages had to be ignored consecutively
Se